In [72]:
# 将生成的数据作为低保真数据 
from smote_variants import ADASYN, CBSO, SMOTE, SMOTE_PSO, SMOTE_PSOBAT, SMOTE_D, Borderline_SMOTE1, Safe_Level_SMOTE, Random_SMOTE
import pandas as pd
import os
import numpy as np
import sklearn.datasets as datasets
from sklearn.model_selection import KFold,train_test_split   #交叉验证
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score,roc_curve,auc,f1_score, precision_score,confusion_matrix
from sklearn.metrics import precision_recall_curve
import numpy as np
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
#基于提升的方法
from sklearn.ensemble import AdaBoostClassifier
#决策树回归
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA

In [5]:
def spe(Y_test,Y_pred,n):
    
    spe = []
    con_mat = confusion_matrix(Y_test,Y_pred)
    for i in range(n):
        number = np.sum(con_mat[:,:])
        tp = con_mat[i][i]
        fn = np.sum(con_mat[i,:]) - tp
        fp = np.sum(con_mat[:,i]) - tp
        tn = number - tp - fn - fp
        spe1 = tn / (tn + fp)
        spe.append(spe1)
    
    return spe

In [6]:
model_list = []
model = ADASYN()
model_list.append(model)
model = CBSO()
model_list.append(model)
model = SMOTE()
model_list.append(model)
model = SMOTE_PSO()
model_list.append(model)
model = SMOTE_PSOBAT()
model_list.append(model)
model = SMOTE_D()
model_list.append(model)
model = Borderline_SMOTE1()
model_list.append(model)
model = Safe_Level_SMOTE()
model_list.append(model)
model = Random_SMOTE()
model_list.append(model)


In [180]:
def getData(X_all,Y_all,x,y):
    class_0 = np.where(y == 0)[0]
    class_1 = np.where(y == 1)[0]
    
    
    #返回低保真跟高保真的分割
    X_l = X_all[len(x):].tolist()
    Y_l = Y_all[len(x):].tolist()
    X_h = []
    Y_h = []
    
    num_l = len(X_l)
    if Y_l[0] == 0:
        tol_num_l = num_l+len(class_1)-len(class_0)
    else:
        tol_num_l = num_l+len(class_0)-len(class_1)

    print(num_l,len(class_0),len(class_1))    
    for i in range(len(x)):
        if len(X_l) == num_l*2:
            X_h.append(x[i])
            Y_h.append(y[i])
        else:    
            if y[i] != Y_l[0]:
                X_l.append(x[i])
                Y_l.append(y[i])
            else:
                X_h.append(x[i])
                Y_h.append(y[i])
    
    
    return X_l,Y_l,X_h,Y_h

def data_transformer(model,n_estimators,data_x):
    # 使用AdaBoost模型对x进行预测，得到G（x）
    # 一个数据(x1,x2) 经过一个迭代器会产生一个对应的y，然后现在100个迭代器，就产生了(y1,y2,y3....,y100)
    Gx = []
    list_model = model.estimators_
    print(len(list_model))
    for i in range(len(list_model)):
        model = list_model[i]
        Y_hp = model.predict(data_x)
        Gx.append(Y_hp)
    Gx = np.array(Gx)
    #将x与Gx合并在一起
    #将x与上文的迭代器的数据进行合并  即现在的x=(x1,x2,y1,y2,...,y20)
    #np.transpose函数  转置
    Gx_T = Gx.transpose()
    #Gx_T 的shape为（1284，20）
    X_h = np.array(data_x)
    #print(X_h.shape,Gx_T.shape)
    X_new = np.concatenate([X_h,Gx_T], axis=1)
  
    return X_new

In [171]:
path = 'D:\\song\\X面向数据的方法\\dataset\\'
folder_name = 'dataset1_5_fold'
data_list = []
res ={'acu':[],'f1':[],'gmean':[]}

folder_list = os.listdir(path+folder_name)
no_new = []
oversampler = model_list[2]
folder = 'segment0'

data = 0
file_list = os.listdir(path+folder_name+'\\'+folder)
file_path = path+folder_name+'\\'+folder +'\\'+file_list[0]
print(file_path)
skiprows = 12
len_data_list = len(data_list)
over = 0
for i in range(50):
    #print(i)
    if  over == 1:
        break
    try:
        data = pd.read_csv(file_path,skiprows=skiprows,header = None)
        print('read data over')
        over = 1
    except:
        skiprows = skiprows+1


acc = []
f_score= []
gmean_list = []

for i in range(5):
    train_name = path+folder_name+'\\'+folder +'\\' +folder+'-5-'+str(i+1)+'tra.dat'
    test_name = path+folder_name+'\\'+folder +'\\' +folder+'-5-'+str(i+1)+'tst.dat'

    train_data = pd.read_csv(train_name,skiprows=skiprows,header = None)
    test_data = pd.read_csv(test_name,skiprows=skiprows,header = None)

    train_data_x = train_data.iloc[:,:-1]
    train_data_class = train_data.iloc[:,-1]
    test_data_x = test_data.iloc[:,:-1]
    test_data_class = test_data.iloc[:,-1]
    train_data_y = [1 if 'positive' in data else 0 for data in train_data_class]  
    test_data_y = [1 if 'positive' in data else 0 for data in test_data_class]  
    train_data_x = np.array(train_data_x)
    train_data_y = np.array(train_data_y)
    test_data_x = np.array(test_data_x)
    test_data_y = np.array(test_data_y)


    X_samp, y_samp= oversampler.sample(train_data_x, train_data_y)  #输入的X是一个二维数据  y是一个一维数组
    print(len(X_samp), len(y_samp),len(train_data_x),len(train_data_y))
    if len(X_samp) == len(train_data_x):
        no_new.append(str(model)+str(train_name))
        break
    #构建低保真模型
    X_l,Y_l,X_h,Y_h= getData(X_samp, y_samp, train_data_x, train_data_y)
    
    
    n_estimators = 20
    model = AdaBoostClassifier(n_estimators=n_estimators, random_state=1)
    model.fit(X_l, Y_l)
    X_h_new = data_transformer(model,n_estimators,X_h)
    #pca = PCA(n_components=len(X_l[0]))
    #pca.fit(X_h_new)
    #X_h_p = pca.transform(X_h_new)
    
    
    
    #X_new = X_samp[len(train_data_x):]
    
    svc = svm.SVC()
    #print(x_train.shape, y_train.shape, x_test.shape)
    svc.fit(X_h_new, Y_h)
    #y_pre = svc.decision_function(x_test)
    
    X_test_new = data_transformer(model,n_estimators,test_data_x)
    #pca = PCA(n_components=len(X_l))
    #pca.fit(X_h_new)
    #X_test_p = pca.transform(X_test_new)
    
    y_pre = svc.predict(X_test_new)

    #print(y_test,y_pre)
    precision = precision_score(test_data_y, y_pre)
    specificity = spe(test_data_y, y_pre,1)[0]
    #print(precision,specificity)
    gmean_list.append(np.sqrt(precision*specificity))
    # 返回三个数组结果分别是fpr(假正率),tpr(召回率),threshold(阈值)
    # 参数为真实结果数据、预测结果数据
    fpr, tpr, threshold = roc_curve(test_data_y, y_pre)
    #print(fpr, tpr)
    # 计算AUC的值

    #print(auc(fpr, tpr))
    acc.append(roc_auc_score(test_data_y, y_pre))
    f_score.append(f1_score(test_data_y, y_pre)) #F1的值
    #print(roc_auc_score(y_test, y_pre))#计算曲线ROC的面积  AUC的值
res['acu'].append(np.mean(acc))#平均ROC
res['f1'].append(np.mean(f_score))#F1的平均值
res['gmean'].append(np.mean(gmean_list))#gmean的平均值
#print(res)

2022-11-12 17:15:55,231:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:15:55,232:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:15:55,233:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:15:55,245:INFO:SMOTE: simplex sampling with n_dim 2


D:\song\X面向数据的方法\dataset\dataset1_5_fold\segment0\segment0-5-1tra.dat
read data over
3166 3166 1846 1846
440 1583 263
20


2022-11-12 17:15:55,433:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:15:55,434:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:15:55,435:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:15:55,451:INFO:SMOTE: simplex sampling with n_dim 2


20
3166 3166 1846 1846
440 1583 263
20
20


2022-11-12 17:15:55,666:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:15:55,668:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:15:55,670:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:15:55,690:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:15:55,884:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:15:55,885:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:15:55,886:INFO:NearestNeighborsW

3166 3166 1846 1846
440 1583 263
20
20


2022-11-12 17:15:56,108:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")


3166 3166 1847 1847
441 1583 264
20
20


2022-11-12 17:15:56,110:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:15:56,111:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:15:56,130:INFO:SMOTE: simplex sampling with n_dim 2


3168 3168 1847 1847
441 1584 263
20
20


In [172]:
print(len(X_l),len(Y_l),len(X_h),len(Y_h),res)

882 882 1406 1406 {'acu': [0.8713675213675215], 'f1': [0.8405192441253823], 'gmean': [0.8478086301506937]}


In [140]:
path = 'D:\\song\\X面向数据的方法\\dataset\\'
folder_name = 'dataset1_5_fold'
data_list = []
res ={'acu':[],'f1':[],'gmean':[]}

folder_list = os.listdir(path+folder_name)

oversampler = model_list[2]
folder = 'segment0'

data = 0
file_list = os.listdir(path+folder_name+'\\'+folder)
file_path = path+folder_name+'\\'+folder +'\\'+file_list[0]
print(file_path)
skiprows = 12
len_data_list = len(data_list)
over = 0
for i in range(50):
    if  over == 1:
        break
    try:
        data = pd.read_csv(file_path,skiprows=skiprows,header = None)
        print('read data over')
        over = 1
    except:
        skiprows = skiprows+1


oversampler= model_list[2]
acc = []
f_score= []
gmean_list = []
for i in range(5):
    train_name = path+folder_name+'\\'+folder +'\\' +folder+'-5-'+str(i+1)+'tra.dat'
    test_name = path+folder_name+'\\'+folder +'\\' +folder+'-5-'+str(i+1)+'tst.dat'

    train_data = pd.read_csv(train_name,skiprows=skiprows,header = None)
    test_data = pd.read_csv(test_name,skiprows=skiprows,header = None)

    train_data_x = train_data.iloc[:,:-1]
    train_data_class = train_data.iloc[:,-1]
    test_data_x = test_data.iloc[:,:-1]
    test_data_class = test_data.iloc[:,-1]
    train_data_y = [1 if 'positive' in data else 0 for data in train_data_class]  
    test_data_y = [1 if 'positive' in data else 0 for data in test_data_class]  
    train_data_x = np.array(train_data_x)
    train_data_y = np.array(train_data_y)
    test_data_x = np.array(test_data_x)
    test_data_y = np.array(test_data_y)


    X_samp, y_samp= oversampler.sample(train_data_x, train_data_y)  #输入的X是一个二维数据  y是一个一维数组

    svc = svm.SVC()
    #print(x_train.shape, y_train.shape, x_test.shape)
    svc.fit(X_samp, y_samp)


    y_pre = svc.predict(test_data_x)

    #print(y_test,y_pre)
    precision = precision_score(test_data_y, y_pre)
    specificity = spe(test_data_y, y_pre,1)[0]
    #print(precision,specificity)
    gmean_list.append(np.sqrt(precision*specificity))
    # 返回三个数组结果分别是fpr(假正率),tpr(召回率),threshold(阈值)
    # 参数为真实结果数据、预测结果数据
    fpr, tpr, threshold = roc_curve(test_data_y, y_pre)
    #print(fpr, tpr)
    # 计算AUC的值

    #print(auc(fpr, tpr))
    acc.append(roc_auc_score(test_data_y, y_pre))
    f_score.append(f1_score(test_data_y, y_pre)) #F1的值
    #print(roc_auc_score(y_test, y_pre))#计算曲线ROC的面积  AUC的值
res['acu'].append(np.mean(acc))#平均ROC
res['f1'].append(np.mean(f_score))#F1的平均值
res['gmean'].append(np.mean(gmean_list))#gmean的平均值
print(res)

2022-11-12 17:07:50,829:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:07:50,830:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:07:50,831:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:07:50,849:INFO:SMOTE: simplex sampling with n_dim 2


D:\song\X面向数据的方法\dataset\dataset1_5_fold\segment0\segment0-5-1tra.dat
read data over


2022-11-12 17:07:51,114:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:07:51,116:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:07:51,117:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:07:51,133:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:07:51,423:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:07:51,425:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:07:51,426:INFO:NearestNeighborsW

{'acu': [0.9838339087073263], 'f1': [0.9501259827602538], 'gmean': [0.9506639623092796]}


In [139]:
##   使用一种算法对所有数据集做评价
##


path = 'D:\\song\\X面向数据的方法\\dataset\\'
folder_name = 'dataset1_5_fold'
data_list = []
res ={'acu':[],'f1':[],'gmean':[]}

folder_list = os.listdir(path+folder_name)

oversampler = model_list[2]
folder = 'segment0'

for folder in folder_list:
    data = 0
    file_list = os.listdir(path+folder_name+'\\'+folder)
    file_path = path+folder_name+'\\'+folder +'\\'+file_list[0]
    print(file_path)
    skiprows = 12
    len_data_list = len(data_list)
    over = 0
    for i in range(50):
        print(i)
        if  over == 1:
            break
        try:
            data = pd.read_csv(file_path,skiprows=skiprows,header = None)
            print('read data over')
            over = 1
        except:
            skiprows = skiprows+1
        
    
    oversampler= model_list[2]
    acc = []
    f_score= []
    gmean_list = []
    for i in range(5):
        train_name = path+folder_name+'\\'+folder +'\\' +folder+'-5-'+str(i+1)+'tra.dat'
        test_name = path+folder_name+'\\'+folder +'\\' +folder+'-5-'+str(i+1)+'tst.dat'

        train_data = pd.read_csv(train_name,skiprows=skiprows,header = None)
        test_data = pd.read_csv(test_name,skiprows=skiprows,header = None)

        train_data_x = train_data.iloc[:,:-1]
        train_data_class = train_data.iloc[:,-1]
        test_data_x = test_data.iloc[:,:-1]
        test_data_class = test_data.iloc[:,-1]
        train_data_y = [1 if 'positive' in data else 0 for data in train_data_class]  
        test_data_y = [1 if 'positive' in data else 0 for data in test_data_class]  
        train_data_x = np.array(train_data_x)
        train_data_y = np.array(train_data_y)
        test_data_x = np.array(test_data_x)
        test_data_y = np.array(test_data_y)


        X_samp, y_samp= oversampler.sample(train_data_x, train_data_y)  #输入的X是一个二维数据  y是一个一维数组

        svc = svm.SVC()
        #print(x_train.shape, y_train.shape, x_test.shape)
        svc.fit(X_samp, y_samp)


        y_pre = svc.predict(test_data_x)

        #print(y_test,y_pre)
        precision = precision_score(test_data_y, y_pre)
        specificity = spe(test_data_y, y_pre,1)[0]
        #print(precision,specificity)
        gmean_list.append(np.sqrt(precision*specificity))
        # 返回三个数组结果分别是fpr(假正率),tpr(召回率),threshold(阈值)
        # 参数为真实结果数据、预测结果数据
        fpr, tpr, threshold = roc_curve(test_data_y, y_pre)
        #print(fpr, tpr)
        # 计算AUC的值

        #print(auc(fpr, tpr))
        acc.append(roc_auc_score(test_data_y, y_pre))
        f_score.append(f1_score(test_data_y, y_pre)) #F1的值
        #print(roc_auc_score(y_test, y_pre))#计算曲线ROC的面积  AUC的值
    res['acu'].append(np.mean(acc))#平均ROC
    res['f1'].append(np.mean(f_score))#F1的平均值
    res['gmean'].append(np.mean(gmean_list))#gmean的平均值
print(res)

2022-11-12 17:06:15,513:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:06:15,514:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:06:15,515:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:06:15,517:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:06:15,527:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:06:15,529:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:06:15,531:INFO:NearestNeighborsW

D:\song\X面向数据的方法\dataset\dataset1_5_fold\ecoli-0_vs_1\ecoli-0_vs_1-5-1tra.dat
0
read data over
1
D:\song\X面向数据的方法\dataset\dataset1_5_fold\ecoli1\ecoli1-5-1tra.dat
0
read data over
1
D:\song\X面向数据的方法\dataset\dataset1_5_fold\ecoli2\ecoli2-5-1tra.dat
0
read data over
1


2022-11-12 17:06:15,688:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:06:15,689:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:06:15,690:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:06:15,702:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:06:15,703:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:06:15,704:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:06:15,706:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:06:15,720:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', '

D:\song\X面向数据的方法\dataset\dataset1_5_fold\ecoli3\ecoli3-5-1tra.dat
0
read data over
1
D:\song\X面向数据的方法\dataset\dataset1_5_fold\glass-0-1-2-3_vs_4-5-6\glass-0-1-2-3_vs_4-5-6-5-1tra.dat
0
1
2
read data over
3
D:\song\X面向数据的方法\dataset\dataset1_5_fold\glass0\glass0-5-1tra.dat
0
1
2
read data over
3


2022-11-12 17:06:15,914:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:06:15,915:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:06:15,917:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:06:15,934:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:06:15,935:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:06:15,936:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:06:15,937:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:06:15,950:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', '

D:\song\X面向数据的方法\dataset\dataset1_5_fold\glass1\glass1-5-1tra.dat
0
1
2
read data over
3
D:\song\X面向数据的方法\dataset\dataset1_5_fold\glass6\glass6-5-1tra.dat
0
1
2
read data over
3
D:\song\X面向数据的方法\dataset\dataset1_5_fold\haberman\haberman-5-1tra.dat
0
read data over
1


2022-11-12 17:06:16,143:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:06:16,145:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:06:16,146:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:06:16,161:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:06:16,161:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:06:16,162:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:06:16,163:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:06:16,179:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', '

D:\song\X面向数据的方法\dataset\dataset1_5_fold\iris0\iris0-5-1tra.dat
0
read data over
1
D:\song\X面向数据的方法\dataset\dataset1_5_fold\new-thyroid1\new-thyroid1-5-1tra.dat
0
read data over
1
D:\song\X面向数据的方法\dataset\dataset1_5_fold\page-blocks0\page-blocks0-5-1tra.dat
0
1
2
3
read data over
4


2022-11-12 17:06:19,799:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:06:19,800:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:06:19,802:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:06:19,804:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:06:27,203:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:06:27,204:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:06:27,206:INFO:NearestNeighborsW

KeyboardInterrupt: 

In [120]:
res

{'acu': [0.8451048074732286],
 'f1': [0.6925390551477508],
 'gmean': [0.7018438235304234]}

In [181]:
path = 'D:\\song\\X面向数据的方法\\dataset\\'
folder_name = 'dataset1_5_fold'
data_list = []
res ={'acu':[],'f1':[],'gmean':[]}

folder_list = os.listdir(path+folder_name)

no_new = []

for folder in folder_list:
    data = 0
    file_list = os.listdir(path+folder_name+'\\'+folder)
    file_path = path+folder_name+'\\'+folder +'\\'+file_list[0]
    print(file_path)
    skiprows = 12
    len_data_list = len(data_list)
    over = 0
    for i in range(50):
        print(i)
        if  over == 1:
            break
        try:
            data = pd.read_csv(file_path,skiprows=skiprows,header = None)
            print('read data over')
            over = 1
        except:
            skiprows = skiprows+1
        
    
    oversampler= model_list[2]
    acc = []
    f_score= []
    gmean_list = []
    for i in range(5):
        print(model,i)
        train_name = path+folder_name+'\\'+folder +'\\' +folder+'-5-'+str(i+1)+'tra.dat'
        test_name = path+folder_name+'\\'+folder +'\\' +folder+'-5-'+str(i+1)+'tst.dat'

        train_data = pd.read_csv(train_name,skiprows=skiprows,header = None)
        test_data = pd.read_csv(test_name,skiprows=skiprows,header = None)

        train_data_x = train_data.iloc[:,:-1]
        train_data_class = train_data.iloc[:,-1]
        test_data_x = test_data.iloc[:,:-1]
        test_data_class = test_data.iloc[:,-1]
        train_data_y = [1 if 'positive' in data else 0 for data in train_data_class]  
        test_data_y = [1 if 'positive' in data else 0 for data in test_data_class]  
        train_data_x = np.array(train_data_x)
        train_data_y = np.array(train_data_y)
        test_data_x = np.array(test_data_x)
        test_data_y = np.array(test_data_y)

        X_samp, y_samp= oversampler.sample(train_data_x, train_data_y)  #输入的X是一个二维数据  y是一个一维数组

        if len(X_samp) == len(train_data_x):
            no_new.append(str(model)+str(train_name))
            break
        #构建低保真模型
        X_l,Y_l,X_h,Y_h= getData(X_samp, y_samp, train_data_x, train_data_y)

        n_estimators = 5
        model = AdaBoostClassifier(
        n_estimators=n_estimators, random_state=5
    )
        model.fit(X_l, Y_l)
        #print(model.estimators_)
        X_h_new = data_transformer(model,n_estimators,X_h)
        #pca = PCA(n_components=len(X_l[0]))
        #pca.fit(X_h_new)
        #X_h_p = pca.transform(X_h_new)



        #X_new = X_samp[len(train_data_x):]

        svc = svm.SVC()
        #print(x_train.shape, y_train.shape, x_test.shape)
        svc.fit(X_h_new, Y_h)
        #y_pre = svc.decision_function(x_test)

        X_test_new = data_transformer(model,n_estimators,test_data_x)
        #pca = PCA(n_components=len(X_l))
        #pca.fit(X_h_new)
        #X_test_p = pca.transform(X_test_new)

        y_pre = svc.predict(X_test_new)

        #print(y_test,y_pre)
        precision = precision_score(test_data_y, y_pre)
        specificity = spe(test_data_y, y_pre,1)[0]
        #print(precision,specificity)
        gmean_list.append(np.sqrt(precision*specificity))
        # 返回三个数组结果分别是fpr(假正率),tpr(召回率),threshold(阈值)
        # 参数为真实结果数据、预测结果数据
        fpr, tpr, threshold = roc_curve(test_data_y, y_pre)
        #print(fpr, tpr)
        # 计算AUC的值

        #print(auc(fpr, tpr))
        acc.append(roc_auc_score(test_data_y, y_pre))
        f_score.append(f1_score(test_data_y, y_pre)) #F1的值


    #print(roc_auc_score(y_test, y_pre))#计算曲线ROC的面积  AUC的值
    res['acu'].append(np.mean(acc))#平均ROC
    res['f1'].append(np.mean(f_score))#F1的平均值
    res['gmean'].append(np.mean(gmean_list))#gmean的平均值
    

2022-11-12 17:21:37,294:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:21:37,295:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:21:37,297:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:21:37,299:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:21:37,322:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:21:37,323:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:21:37,324:INFO:NearestNeighborsW

D:\song\X面向数据的方法\dataset\dataset1_5_fold\ecoli-0_vs_1\ecoli-0_vs_1-5-1tra.dat
0
read data over
1
AdaBoostClassifier(n_estimators=5, random_state=5) 0
52 62 114
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 1
52 62 114
1
1
AdaBoostClassifier(n_estimators=5, random_state=5) 2
52 62 114
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 3
54 61 115
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 4
54 61 115
5
5
D:\song\X面向数据的方法\dataset\dataset1_5_fold\ecoli1\ecoli1-5-1tra.dat
0
read data over
1
AdaBoostClassifier(n_estimators=5, random_state=5) 0
146 207 61
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 1
147 208 61
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 2
145 207 62
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 3


2022-11-12 17:21:37,483:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:21:37,485:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:21:37,486:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:21:37,510:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:21:37,511:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:21:37,512:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:21:37,514:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:21:37,542:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', '

145 207 62
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 4
145 207 62
5
5
D:\song\X面向数据的方法\dataset\dataset1_5_fold\ecoli2\ecoli2-5-1tra.dat
0
read data over
1
AdaBoostClassifier(n_estimators=5, random_state=5) 0
186 227 41
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 1
187 228 41
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 2
185 227 42
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 3
185 227 42
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 4
185 227 42
5
5
D:\song\X面向数据的方法\dataset\dataset1_5_fold\ecoli3\ecoli3-5-1tra.dat
0
read data over
1
AdaBoostClassifier(n_estimators=5, random_state=5) 0
212 240 28
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 1


2022-11-12 17:21:37,672:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:21:37,690:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:21:37,691:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:21:37,692:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:21:37,693:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:21:37,709:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:21:37,710:INFO:NearestNeighborsWithMetricTensor: NN fitting wi

...

164 355 191
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 2
164 355 191
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 3
163 355 192
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 4
165 356 191
5
5
D:\song\X面向数据的方法\dataset\dataset1_5_fold\yeast1\yeast1-5-1tra.dat
0
1
read data over
2
AdaBoostClassifier(n_estimators=5, random_state=5) 0
501 844 343
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 1
501 844 343
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 2


2022-11-12 17:21:41,693:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:21:41,694:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:21:41,697:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:21:41,746:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:21:41,746:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:21:41,748:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:21:41,750:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:21:41,826:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', '

501 844 343
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 3
501 844 343
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 4
500 844 344
5
5
D:\song\X面向数据的方法\dataset\dataset1_5_fold\yeast3\yeast3-5-1tra.dat
0
1
read data over
2
AdaBoostClassifier(n_estimators=5, random_state=5) 0


2022-11-12 17:21:41,898:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:21:41,899:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:21:41,901:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:21:41,929:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2022-11-12 17:21:41,932:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2022-11-12 17:21:41,933:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2022-11-12 17:21:41,935:INFO:SMOTE: simplex sampling with n_dim 2
2022-11-12 17:21:41,964:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', '

927 1057 130
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 1
927 1057 130
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 2
927 1057 130
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 3
925 1056 131
5
5
AdaBoostClassifier(n_estimators=5, random_state=5) 4
926 1057 131
5
5


In [182]:
res

{'acu': [0.9764285714285714,
  0.8958936651583711,
  0.6775102529049897,
  0.8924160811865731,
  0.8497348484848486,
  0.6979064039408867,
  0.5781150793650793,
  0.931081081081081,
  0.5985272767643642,
  1.0,
  0.8333053221288516,
  0.5813654803991029,
  0.7109706498951781,
  0.8799639038879545,
  0.7774001192605844,
  0.6637582469210377,
  0.6165994764924998,
  0.6557472467104404,
  0.9766623472352016,
  0.6891440667526793,
  0.9281664522584334],
 'f1': [0.9861941448382125,
  0.7787128037128037,
  0.3893668236525379,
  0.6077639751552795,
  0.7710754604872252,
  0.617765332693586,
  0.5503797263586285,
  0.8451881451881451,
  0.36448610326871195,
  1.0,
  0.6924297924297924,
  0.21466345419867947,
  0.6278838689644466,
  0.5829216604113475,
  0.5809605247571665,
  0.5011847817865517,
  0.4682121089962739,
  0.48622845506705514,
  0.9614495980210267,
  0.5603872121020493,
  0.7603537779126375],
 'gmean': [0.9863113493624981,
  0.791323203367899,
  0.4582140080349556,
  0.647761846992